# Exercício 6

A Metalúrgica Araucária S/A, dentro de 60 dias, deverá começar a funcionar em
sua nova sede localizada na Cidade Industrial de Curitiba (CIC). O Presidente
da Metalúrgica deseja que a distribuição das salas, dessa nova instalação, seja
feita de modo a atender, na medida do possível, as preferências já manifestadas.
Em uma pesquisa realizada, os Diretores manifestaram as suas preferências:

| **Diretor/Sala** 	| **Sala 1** 	| **Sala 2** 	| **Sala 3** 	| **Sala 4** 	| **Sala 5** 	| **Sala 6** 	|
|:----------------:	|:----------:	|:----------:	|:----------:	|:----------:	|:----------:	|:----------:	|
|   **Diretor 1**  	|      2     	|      4     	|      3     	|      1     	|      5     	|      6     	|
|   **Diretor 2**  	|      1     	|      5     	|      4     	|      6     	|      3     	|      2     	|
|   **Diretor 3**  	|      5     	|      3     	|      4     	|      2     	|      1     	|      6     	|
|   **Diretor 4**  	|      1     	|      3     	|      2     	|      4     	|      6     	|      5     	|
|   **Diretor 5**  	|      3     	|      2     	|      5     	|      6     	|      1     	|      3     	|

Se você fosse convidado a opinar sobre a distribuição das salas qual seria a sua recomendação?

# Solução

Trata-se de um problema de designação desbalanceado, contendo mais salas do que diretores.

## Modelo Matemático

### Função Objetivo

$$ min \ Z = \sum_{i \in I} \sum_{j \in J} c_{ij}x_{ij} $$

### Sujeito a

$$
\sum_{i \in I} x_{ij} \le 1, \ \forall j \in J \ \dots \ (1)
$$

$$
\sum_{j \in J} x_{ij} = 1, \ \forall i \in I \ \dots \ (2)
$$

$$
x_{ij} \in \set{0, 1}, \forall i \in I, \forall j \in J \ \dots \ (3)
$$

In [1]:
# --- Imports --- #
import pyomo.environ as pyo
import numpy as np
import pandas as pd

In [3]:
# --- Declaração dos Dados de Entrada --- #
# Matriz de Preferências
matriz_preferencias = np.array([[2, 4, 3, 1, 5, 6],
                                [1, 5, 4, 6, 3, 2],
                                [5, 3, 4, 2, 1, 6],
                                [1, 3, 2, 4, 6, 5],
                                [3, 2, 5, 6, 1, 3]])
lista_diretores = ['D1', 'D2', 'D3', 'D4', 'D5']
lista_salas = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
custo_preferencia = {(lista_diretores[i], lista_salas[j]): matriz_preferencias[i, j] for i in range(len(lista_diretores))
                     for j in range(len(lista_salas))} # Custo de preferência de cada diretor por sala

In [15]:
# --- Declaração do Modelo Matemático --- #
modelo = pyo.ConcreteModel() # Objeto de modelo concreto do Pyomo

In [16]:
# --- Declaração dos Conjunto de Iteração do Modelo --- #
modelo.diretores = pyo.Set(initialize=lista_diretores) # Conjunto de diretores
modelo.salas = pyo.Set(initialize=lista_salas) # Conjunto de salas

In [17]:
# --- Declaração dos Parâmetros do Modelo --- #
modelo.custo_performance = pyo.Param(modelo.diretores, modelo.salas, initialize=custo_preferencia) # Custo de preferência de cada diretor por sala

In [18]:
# --- Declaração das Variáveis de Decisão do Modelo --- #
modelo.x = pyo.Var(modelo.diretores, modelo.salas, domain=pyo.Binary) # Variável binária de alocação de diretores nas salas

In [19]:
# --- Declaração da Função Objetivo do Modelo --- #
def funcao_objetivo(modelo):
    return sum(modelo.custo_performance[i, j] * modelo.x[i, j] 
               for i in modelo.diretores for j in modelo.salas)
# ---
modelo.objetivo = pyo.Objective(rule=funcao_objetivo, sense=pyo.minimize) # Função objetivo de minimização

In [20]:
# --- Declaração das Restrições do Modelo --- #
# -- Restrição de alocação de diretores em salas -- #
def restricao_diretores(modelo, i):
    '''
    Cada um dos diretores deve ser alocado em uma única sala.
    '''
    return sum(modelo.x[i, :]) == 1
modelo.rest_diretores = pyo.Constraint(modelo.diretores, rule=restricao_diretores)
# ---
# -- Restrição de alocação de salas para diretores -- #
def restricao_salas(modelo, j):
    '''
    Cada uma das salas deve ser alocada para um único diretor, exceto as que excedam o número
    de diretores.
    '''
    return sum(modelo.x[:, j]) <= 1
modelo.rest_salas = pyo.Constraint(modelo.salas, rule=restricao_salas)

In [21]:
modelo.pprint()

2 Set Declarations
    diretores : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'D1', 'D2', 'D3', 'D4', 'D5'}
    salas : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'S1', 'S2', 'S3', 'S4', 'S5', 'S6'}

1 Param Declarations
    custo_performance : Size=30, Index=diretores*salas, Domain=Any, Default=None, Mutable=False
        Key          : Value
        ('D1', 'S1') :     2
        ('D1', 'S2') :     4
        ('D1', 'S3') :     3
        ('D1', 'S4') :     1
        ('D1', 'S5') :     5
        ('D1', 'S6') :     6
        ('D2', 'S1') :     1
        ('D2', 'S2') :     5
        ('D2', 'S3') :     4
        ('D2', 'S4') :     6
        ('D2', 'S5') :     3
        ('D2', 'S6') :     2
        ('D3', 'S1') :     5
        ('D3', 'S2') :     3
        ('D3', 'S3') :     4
        ('D3', 'S4') :     2
        ('D3', 'S5') :    

In [22]:
modelo.write('modelo.lp', io_options={'symbolic_solver_labels': True})

('modelo.lp', 2963884584464)

In [23]:
# --- Declaração do Solver --- # 
solver = pyo.SolverFactory('gurobi') # Instanciação do solver Gurobi
solver.solve(modelo) # Resolução do modelo

{'Problem': [{'Name': 'x1', 'Lower bound': 7.0, 'Upper bound': 7.0, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 30, 'Number of binary variables': 30, 'Number of integer variables': 30, 'Number of continuous variables': 0, 'Number of nonzeros': 60, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.003999948501586914', 'Error rc': 0, 'Time': 0.9265580177307129}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [24]:
print(f'Função Objetivo: {modelo.objetivo()}')

Função Objetivo: 7.0


In [25]:
modelo.x.display() # Exibição da alocação de diretores nas salas

x : Size=30, Index=diretores*salas
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
    ('D1', 'S1') :     0 :  -0.0 :     1 : False : False : Binary
    ('D1', 'S2') :     0 :  -0.0 :     1 : False : False : Binary
    ('D1', 'S3') :     0 :  -0.0 :     1 : False : False : Binary
    ('D1', 'S4') :     0 :   1.0 :     1 : False : False : Binary
    ('D1', 'S5') :     0 :  -0.0 :     1 : False : False : Binary
    ('D1', 'S6') :     0 :  -0.0 :     1 : False : False : Binary
    ('D2', 'S1') :     0 :   1.0 :     1 : False : False : Binary
    ('D2', 'S2') :     0 :  -0.0 :     1 : False : False : Binary
    ('D2', 'S3') :     0 :  -0.0 :     1 : False : False : Binary
    ('D2', 'S4') :     0 :  -0.0 :     1 : False : False : Binary
    ('D2', 'S5') :     0 :  -0.0 :     1 : False : False : Binary
    ('D2', 'S6') :     0 :  -0.0 :     1 : False : False : Binary
    ('D3', 'S1') :     0 :  -0.0 :     1 : False : False : Binary
    ('D3', 'S2') :     0 :  -0.0 :     1 

In [27]:
# --- Extração dos Resultados --- #
dados_designacao = [{'Diretor': i, 'Sala': j, 'Designado': val}
                    for (i, j), val in modelo.x.extract_values().items()]
# ---
resultados = pd.DataFrame(dados_designacao).pivot(index='Diretor',
                                                  columns='Sala',
                                                  values='Designado')
resultados.to_excel('_ex_06_resultados_modelo_matematico.xlsx')